## Part1: Data

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
i=0
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        i=i+1

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd 
import numpy as np 
import tensorflow as tf
import matplotlib.pyplot as plt 
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
import os
import time
import math
from tensorflow.keras.applications.resnet_v2 import ResNet152V2
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Input, Dropout
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam

In [ ]:
PATH = "../input/applications-of-deep-learning-wustlfall-2021/city"
PATH_TRAIN = os.path.join(PATH, "train.csv")
PATH_TEST = os.path.join(PATH, "test.csv")

In [ ]:
df_train = pd.read_csv(PATH_TRAIN)
df_test = pd.read_csv(PATH_TEST)

df_train['filename'] = df_train.id.astype(str) + ".jpg"
df_test['filename'] = df_test.id.astype(str) + ".jpg"

In [ ]:
df_train.head()

In [ ]:
df_test.head()

In [ ]:
img = Image.open('../input/applications-of-deep-learning-wustlfall-2021/city/1.jpg')
#img = img.resize((512,512),Image.ANTIALIAS)
img_arr = np.asarray(img)
fig,ax = plt.subplots(figsize=(6,6))
ax.imshow(img_arr)
ax.set_title('Original Figure')

In [ ]:
#Horizontal/Vertical Flip
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            horizontal_flip = True,
            vertical_flip = True,
            fill_mode = 'nearest')
aug_iter = datagen.flow(np.expand_dims(img_arr,axis=0), batch_size=1)

fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(15,15))
# generate batch of images
for i in range(5):
    image = next(aug_iter)[0].astype('uint8')
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i].set_title('Horizonal/Vertical Flipped')

In [ ]:
#Horizontal/Vertical Shift
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            width_shift_range=0.1, 
            height_shift_range=0.1,
            fill_mode = 'nearest')
aug_iter = datagen.flow(np.expand_dims(img_arr,axis=0), batch_size=1)

fig, ax = plt.subplots(nrows=1,ncols=3,figsize=(15,15))
# generate batch of images
for i in range(3):
    image = next(aug_iter)[0].astype('uint8')
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i].set_title('Vertical/Horizontal Shift')

In [ ]:
#rotation
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            rotation_range = 45,
            fill_mode = 'nearest')
aug_iter = datagen.flow(np.expand_dims(img_arr,axis=0), batch_size=1)

fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(15,15))
# generate batch of images
for i in range(5):
    image = next(aug_iter)[0].astype('uint8')
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i].set_title('Rotation')

In [ ]:
#Brightness
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            brightness_range = [0.8,1.1],
            fill_mode = 'nearest')
aug_iter = datagen.flow(np.expand_dims(img_arr,axis=0), batch_size=1)

fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(15,15))
# generate batch of images
for i in range(5):
    image = next(aug_iter)[0].astype('uint8')
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i].set_title('Brighten')

In [ ]:
#How brightness influence dark pictures
## original pictures
img2 = Image.open('../input/applications-of-deep-learning-wustlfall-2021/city/10072.jpg')
#img = img.resize((IMAGE_WIDTH,IMAGE_HEIGHT),Image.ANTIALIAS)
img2_arr = np.asarray(img2)
fig,ax = plt.subplots(figsize=(6,6))
ax.imshow(img2_arr)
ax.set_title('Original Figure')

In [ ]:
#after brightness
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            brightness_range = [0.8,1.1],
            fill_mode = 'nearest')
aug_iter = datagen.flow(np.expand_dims(img2_arr,axis=0), batch_size=1)

fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(15,15))
# generate batch of images
for i in range(5):
    image = next(aug_iter)[0].astype('uint8')
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i].set_title('Brighten')

In [ ]:
#zoom
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
            zoom_range = [0.5,1.5],
            fill_mode = 'nearest')
aug_iter = datagen.flow(np.expand_dims(img_arr,axis=0), batch_size=1)

fig, ax = plt.subplots(nrows=1,ncols=5,figsize=(15,15))
# generate batch of images
for i in range(5):
    image = next(aug_iter)[0].astype('uint8')
    ax[i].imshow(image)
    ax[i].axis('off')
    ax[i].set_title('Zoom')

## Part2: Data Preparation

In [ ]:
#data preparation
TRAIN_PCT = 0.9
TRAIN_CUT = int(len(df_train) * TRAIN_PCT)

df_train_cut = df_train[0:TRAIN_CUT]
df_validate_cut = df_train[TRAIN_CUT:]

print(f"Training size: {len(df_train_cut)}")
print(f"Validate size: {len(df_validate_cut)}")

In [ ]:
def DataGeneration(Height, Width, BatchSize):
    start_time = time.time()
    training_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1./255,
                horizontal_flip = True,
                vertical_flip = True,
                rotation_range = 30,
                brightness_range = [0.8,1.5],
                zoom_range = [0.8,1.2],
                fill_mode = 'nearest')
    train_generator = training_datagen.flow_from_dataframe(
            dataframe = df_train_cut,
            directory = PATH,
            x_col="filename",
            y_col="sqft",
            target_size = (Height, Width),
            batch_size = BatchSize,
            class_mode = 'raw',
            shuffle = True,
            seed = 98)
    validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
                rescale = 1./255)
    val_generator = validation_datagen.flow_from_dataframe(
            dataframe = df_validate_cut,
            directory = PATH,
            x_col="filename",
            y_col="sqft",
            target_size = (Height, Width),
            batch_size = BATCH_SIZE,
            class_mode = 'raw',
            shuffle = True,
            seed = 98)

    print(f'----{time.time()-start_time} seconds----')
    return train_generator, val_generator

In [ ]:
## Warmup with Expotential Decay Learning Rate Scheduler
def scheduler(epoch):
    if epoch<= WARMUP_EPOCH:
        lr = INITIAL_LEARNINGRATE *epoch/WARMUP_EPOCH
    
    else:
        lr = INITIAL_LEARNINGRATE * DECAY_RATE**(epoch-WARMUP_EPOCH)
    
    return lr

## Part3: Transfer Learning

In [ ]:
#transfer learning
def BuildModel(ModelName):
    print('--------------Building The Model...--------------')
    if ModelName == 'ResNet50' or ModelName == 'Xception':
        if ModelName == 'ResNet50':
            base_model = ResNet50(include_top=False,
                                                    weights='imagenet',
                                                    input_shape=(HEIGHT,WIDTH,3))
        
        elif ModelName == 'Xception':
            base_model = Xception(include_top=False,
                                                weights='imagenet',
                                                input_shape=(HEIGHT,WIDTH,3))
        
        base_model.trainable = True
        print("\nNumber of layers in the base model: ", len(base_model.layers))
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        out = tf.keras.layers.Dense(1)(x)
        model = tf.keras.models.Model(inputs=base_model.input, outputs=out)
        
    
    elif ModelName == 'MobileNet'or ModelName =='VGG16':
        base_model = MobileNet(include_top=False,
                                                weights='imagenet',
                                                input_shape=(HEIGHT,WIDTH,3))
        base_model.trainable = True
        print("\nNumber of layers in the base model: ", len(base_model.layers))
        
        x = base_model.output
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        x = tf.keras.layers.Dense(1024,activation='relu')(x)
        out = tf.keras.layers.Dense(1)(x)
        model = tf.keras.models.Model(inputs=base_model.input, outputs=out)
    
    print('\n--------------Done!--------------')
    return model

In [ ]:
def ModelDeployment(model, LearningRate, ModelName,  NbOfEpoch, StepsPerEpoch, ValidationSteps):
    ## Freeze all the layers before the `fine_tune_at` layer
    #for layer in base_model.layers[:fine_tune_at]:
    #    layer.trainable =  False
    print('--------------Deploying the Model...--------------')
    model.compile(loss='mean_squared_error',optimizer=Adam(learning_rate=LearningRate),metrics = [RootMeanSquaredError(name="rmse")])
    monitor = EarlyStopping(monitor='val_loss',min_delta=1e-4,patience=10,verbose=1,mode='min',restore_best_weights=True)
    #lr_scheduler = tf.keras.callbacks.LearningRateScheduler(scheduler,verbose=1)
    lr_scheduler = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss',
                                                        factor=0.5,
                                                        patience=3,
                                                        verbose=1,
                                                        mode='min',
                                                        min_delta=1e-4,
                                                        cooldown=0,
                                                        min_lr=MIN_LR,)

    filepath= ModelName+"-{epoch:02d}-{val_rmse:.4f}.hdf5"
    checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath,monitor='val_loss',verbose=1,save_best_only=False,save_weights_only=True,mode='min',save_freq = 'epoch')
    print('--------------Deployed Successfully--------------')
    print('--------------Training Begins--------------')
    history = model.fit(train_generator,epochs = NbOfEpoch,steps_per_epoch = StepsPerEpoch,validation_data = val_generator,verbose = 1,validation_steps = ValidationSteps,callbacks = [monitor,lr_scheduler,checkpoint])
    return history

Model1: ResNet50

In [ ]:
#resnet50
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 32

train_generator, val_generator = DataGeneration(Height = HEIGHT, 
                                                Width = WIDTH,
                                                BatchSize = BATCH_SIZE)

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
Resnet_based_model = BuildModel('ResNet50')
Resnet_based_model.summary()

In [ ]:
STEPS_PER_EPOCH = train_generator.n//BATCH_SIZE
VALIDATION_STEPS = val_generator.n//BATCH_SIZE
LEARNING_RATE = 1e-3
NB_EPOCH = 250
MIN_LR = LEARNING_RATE/10
Resnet50_history = ModelDeployment(model = Resnet_based_model, 
                                   LearningRate = LEARNING_RATE, 
                                   ModelName = 'ResNet50',  
                                   NbOfEpoch = NB_EPOCH, 
                                   StepsPerEpoch = STEPS_PER_EPOCH, 
                                   ValidationSteps = VALIDATION_STEPS)

Model2: Xception

In [ ]:
#xception
tf.keras.backend.clear_session()

In [ ]:
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 32

train_generator, val_generator = DataGeneration(Height = HEIGHT, 
                                                Width = WIDTH,
                                                BatchSize = BATCH_SIZE)

In [ ]:
Xception_based_model = BuildModel(ModelName='Xception')
Xception_based_model.summary()

In [ ]:
STEPS_PER_EPOCH = train_generator.n//BATCH_SIZE
VALIDATION_STEPS = val_generator.n//BATCH_SIZE
LEARNING_RATE = 1e-3
NB_EPOCH =50
MIN_LR = 1e-4
Xception_history = ModelDeployment(model = Xception_based_model, 
                                   LearningRate = LEARNING_RATE, 
                                   ModelName = 'Xception',  
                                   NbOfEpoch = NB_EPOCH, 
                                   StepsPerEpoch = STEPS_PER_EPOCH, 
                                   ValidationSteps = VALIDATION_STEPS)

Module3:MobileNet

In [ ]:
#mobilenet
tf.keras.backend.clear_session()

In [ ]:
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 32

train_generator, val_generator = DataGeneration(Height = HEIGHT, 
                                                Width = WIDTH,
                                                BatchSize = BATCH_SIZE)

In [ ]:
MobileNet_based_model = BuildModel(ModelName='MobileNet')
MobileNet_based_model.summary()

In [ ]:
STEPS_PER_EPOCH = train_generator.n//BATCH_SIZE

VALIDATION_STEPS = val_generator.n//BATCH_SIZE

LEARNING_RATE = 1e-4
MIN_LR = LEARNING_RATE/10
NB_EPOCH = 100

MobileNet_history = ModelDeployment(model = MobileNet_based_model, 
                                   LearningRate = LEARNING_RATE, 
                                   ModelName = 'MobileNet',  
                                   NbOfEpoch = NB_EPOCH, 
                                   StepsPerEpoch = STEPS_PER_EPOCH, 
                                   ValidationSteps = VALIDATION_STEPS)

Model4: VGG16

In [ ]:
#vgg16-We finally didn't use it for blending
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 32

train_generator, val_generator = DataGeneration(Height = HEIGHT, 
                                                Width = WIDTH,
                                                BatchSize = BATCH_SIZE)

In [ ]:
vgg16_based_model = BuildModel(ModelName='VGG16')
vgg16_based_model.summary()

In [ ]:
STEPS_PER_EPOCH = train_generator.n//BATCH_SIZE
VALIDATION_STEPS = val_generator.n//BATCH_SIZE
LEARNING_RATE = 1e-3
NB_EPOCH =250
MIN_LR = LEARNING_RATE/10
vgg16_history = ModelDeployment(model = vgg16_based_model, 
                                   LearningRate = LEARNING_RATE, 
                                   ModelName = 'VGG16',  
                                   NbOfEpoch = NB_EPOCH, 
                                   StepsPerEpoch = STEPS_PER_EPOCH, 
                                   ValidationSteps = VALIDATION_STEPS)

## Part 4: Clipping & Blending & Submit

In [ ]:
#submit
HEIGHT = 256
WIDTH = 256
BATCH_SIZE = 32

In [ ]:
Resnet_based_model = BuildModel(ModelName='ResNet50')
Xception_based_model = BuildModel(ModelName='Xception')
MobileNet_based_model = BuildModel(ModelName='MobileNet')
#VGG16_based_model = BuildModel(ModelName='VGG16')

In [ ]:
Resnet_based_model.load_weights('../input/resnet50498/best_ResNet50-38-498.5083.hdf5')
Xception_based_model.load_weights('../input/xception461/Xception-26-461.7788.hdf5')
MobileNet_based_model.load_weights('../input/vgg16-result/VGG16-21-514.5883.hdf5')
#VGG16_based_model.load_weights('../input/1129kaggle4/VGG16-21-514.5883.hdf5')


In [ ]:
submit_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

submit_generator = submit_datagen.flow_from_dataframe(
        dataframe=df_test,
        directory=PATH,
        x_col="filename",
        batch_size = 1,
        shuffle = False,
        target_size=(HEIGHT, WIDTH),
        class_mode=None)

submit_generator.reset()


In [ ]:
Resnet_pred = Resnet_based_model.predict(submit_generator,steps=len(df_test))
Xception_pred = Xception_based_model.predict(submit_generator,steps=len(df_test))
MobileNet_pred = MobileNet_based_model.predict(submit_generator,steps=len(df_test))
#VGG16_pred = VGG16_based_model.predict(submit_generator,steps=len(df_test))

In [ ]:
Resnet_clipped_pred = np.clip(Resnet_pred,np.percentile(Resnet_pred,4),np.percentile(Resnet_pred,96))
Xception_clipped_pred = np.clip(Xception_pred,np.percentile(Xception_pred,3),np.percentile(Xception_pred,97))
MobileNet_clipped_pred = np.clip(MobileNet_pred,np.percentile(MobileNet_pred,4),np.percentile(MobileNet_pred,96))
#VGG16_clipped_val_pred = np.clip(MobileNet_val_pred,np.percentile(VGG16_pred,4),np.percentile(VGG16_pred,96))

ensemble_pred = 0.3*Resnet_clipped_pred + 0.5*Xception_clipped_pred + 0.2*MobileNet_clipped_pred 

In [ ]:
df_submit = pd.DataFrame({"id":df_test['id'],"sqft":ensemble_pred[:,0].flatten()})
df_submit.to_csv("/kaggle/working/4knn_submit_mobilenet_ensamble8.csv",index = False)